<a href="https://colab.research.google.com/github/zhangxs131/Machine_learning_notebook/blob/main/lr_horse_living.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##lr 逻辑回归实战

本次实战内容，将使用Logistic回归来预测患疝气病的马的存活问题。

数据集包含了369个样本（样本数很少）28个特征（特征数很多）最后判断病马是否能存活

原始数据集下载地址：http://archive.ics.uci.edu/ml/datasets/Horse+Colic

原始数据中有很多缺失值，对缺失值进行处理后得到可用数据集


In [3]:
#获取缺失值处理后的数据集

!wget https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Logistic/horseColicTraining.txt
!wget https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Logistic/horseColicTest.txt

--2022-04-30 06:08:38--  https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Logistic/horseColicTraining.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60357 (59K) [text/plain]
Saving to: ‘horseColicTraining.txt’

horseColicTraining. 100%[===================>]  58.94K  --.-KB/s    in 0.01s   

2022-04-30 06:08:38 (5.01 MB/s) - ‘horseColicTraining.txt’ saved [60357/60357]

--2022-04-30 06:08:38--  https://raw.githubusercontent.com/Jack-Cherish/Machine-Learning/master/Logistic/horseColicTest.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, a

In [13]:
#使用改进梯度上升lr算法 随机sample和调整学习率

from os import error
#使用numpy进行lr测试

import numpy as np
import random

def sigmoid(x):
  return 1.0/(1+np.exp(-x))

#改进梯度上升算法

def GradAscent(data,labels,num_iter=150):
  m,n=data.shape
  w=np.ones(n)
  for j in range(num_iter):
    dataIndex=list(range(m))
    for i in range(m):
      alpha=3/(1.0+j+i)+0.01  #逐渐衰减的学习率
      randIndex=int(random.uniform(0,len(dataIndex)))  #随机取样
      h=sigmoid(sum(data[dataIndex[randIndex]]*w))
      loss=labels[dataIndex[randIndex]]-h
      w=w+alpha*loss*data[dataIndex[randIndex]]
      del(dataIndex[randIndex])
  return w

def classifyVector(x,weights):
  prob=sigmoid(sum(x*weights))
  if prob>0.5:
    return 1.0
  else:
    return 0.0



def colTest(train_file,test_file):
  #读取train.txt
  with open(train_file,'r') as f:
    content=f.readlines()
  data=[]
  label=[]
  for i in content:
    t=i.strip().split('\t')
    data.append([float(j) for j in t[:-1]])
    label.append(float(t[-1]))
  weight=GradAscent(np.array(data),label,500)
  errorCount=0.0
  with open(test_file,'r') as f:
    content=f.readlines()
  nums_test=len(content)
  for i in content:
    t=i.strip().split('\t')
    test_x=[float(j) for j in t[:-1]]
    if int(classifyVector(np.array(test_x),weight))!=int(t[-1]):
      errorCount+=1
  error_rate=float(errorCount)/nums_test
  print('错误率为:{:.2f}%'.format(error_rate*100))


colTest('horseColicTraining.txt','horseColicTest.txt')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  


错误率为:32.84%


In [19]:
#使用普通梯度上升算法
def gradAscent(dataMatIn, classLabels,num_iter=150):
    dataMatrix = np.mat(dataMatIn)                                        #转换成numpy的mat
    labelMat = np.mat(classLabels).transpose()                            #转换成numpy的mat,并进行转置
    m, n = np.shape(dataMatrix)                                            #返回dataMatrix的大小。m为行数,n为列数。
    alpha = 0.01                                                        #移动步长,也就是学习速率,控制更新的幅度。                                                       #最大迭代次数
    weights = np.ones((n,1))
    for k in range(num_iter):
        h = sigmoid(dataMatrix * weights)                                #梯度上升矢量化公式
        error = labelMat - h
        weights = weights + alpha * dataMatrix.transpose() * error
    return weights                                               #将矩阵转换为数组，并返回
 
 
def colTest(train_file,test_file):
  #读取train.txt
  with open(train_file,'r') as f:
    content=f.readlines()
  data=[]
  label=[]
  for i in content:
    t=i.strip().split('\t')
    data.append([float(j) for j in t[:-1]])
    label.append(float(t[-1]))
  weight=gradAscent(np.array(data),label,500)
  errorCount=0.0
  with open(test_file,'r') as f:
    content=f.readlines()
  nums_test=len(content)
  for i in content:
    t=i.strip().split('\t')
    test_x=[float(j) for j in t[:-1]]
    if int(classifyVector(np.array(test_x),weight))!=int(t[-1]):
      errorCount+=1
  error_rate=float(errorCount)/nums_test
  print('错误率为:{:.2f}%'.format(error_rate*100))


colTest('horseColicTraining.txt','horseColicTest.txt')

错误率为:28.36%


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in exp
  


#sklearn 进行lr算法

sklearn.linear_model 

*    penalty 惩罚项 L1,L2正则化
*    class_weight 表示标示分类各类型的权重

In [25]:
from sklearn.linear_model import LogisticRegression

def colSklearn(train_file,test_file):
  #读取数据
  with open(train_file,'r') as f:
    content=f.readlines()
  data=[]
  label=[]
  for i in content:
    t=i.strip().split('\t')
    data.append([float(j) for j in t[:-1]])
    label.append(float(t[-1]))
  with open(test_file,'r') as f:
    content=f.readlines()
  test_data=[]
  test_label=[]
  for i in content:
    t=i.strip().split('\t')
    test_data.append([float(j) for j in t[:-1]])
    test_label.append(float(t[-1]))

  #定义LR分类器
  #classifier=LogisticRegression(solver='liblinear',max_iter=100)
  classifier=LogisticRegression(solver='sag',max_iter=5000)
  classifier.fit(data,label)

  test_acc=classifier.score(test_data,test_label)
  print('正确率为{:.3f}%'.format(test_acc*100))

colSklearn('horseColicTraining.txt','horseColicTest.txt')

正确率为73.134%


lr分类器，就是拟合函数，进行二分类，最后通过sigmoid激活，简单易懂，但效果一般